<img src="https://photutils.readthedocs.io/en/latest/_images/photutils_logo_light_plain_path.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

This notebook provides an overview of PSF photometry using [Photutils](https://photutils.readthedocs.io/en/stable/).

# Preliminaries

In [ ]:
# Initial imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import webbpsf
from astropy.table import QTable
from astropy.visualization import simple_norm
from photutils.aperture import CircularAperture
from photutils.datasets import make_noise_image
from photutils.detection import IRAFStarFinder
from photutils.psf import (AiryDiskPSF, CircularGaussianPSF, GaussianPSF,
                           GriddedPSFModel, MoffatPSF, PSFPhotometry,
                           SourceGrouper, make_psf_model_image)
from photutils.utils import make_random_cmap
from tweakwcs import XYXYMatch

# Change some default plotting parameters
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Point Spread Function Photometry with Photutils

The Photutils PSF photometry module provides modular tools that allow users to completely customize the photometry procedure, e.g., by using different source detection algorithms, local background estimators, source groupers, and PSF models. Photutils provides implementations for each subtask involved in the photometry process, however, users are still able to include their own custom implementations.

This modularity is accomplished by using an object oriented programming approach that provides a more convenient user experience.

Photutils provides two top-level classes to perform PSF photometry, `PSFPhotometry` and `IterativelyPSFPhotometry`. In this notebook, we will cover the basics of the `PSFPhotometry` class.

# PSF Models

PSF photometry fundamentally involves fitting models to data. As such, the PSF model is a critical component of PSF photometry. For accurate results, both for photometry and astrometry, the PSF model should be a good representation of the actual data. The PSF model can be a simple analytic function, such as a 2D Gaussian or Moffat profile, or it can be a more complex model derived from a 2D PSF image, e.g., an effective PSF (ePSF). The PSF model can also encapsulate changes in the PSF across the detector, e.g., due to optical aberrations.

Photutils provides the following analytic PSF Models based on the [Astropy Modeling and Fitting framework](https://docs.astropy.org/en/latest/modeling/index.html):

* [GaussianPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GaussianPSF.html#photutils.psf.GaussianPSF)
* [CircularGaussianPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.CircularGaussianPSF.html#photutils.psf.CircularGaussianPSF)
* [MoffatPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.MoffatPSF.html#photutils.psf.MoffatPSF)
* [AiryDiskPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.AiryDiskPSF.html#photutils.psf.AiryDiskPSF)
* [GaussianPRF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GaussianPRF.html#photutils.psf.GaussianPRF)
* [CircularGaussianPRF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.CircularGaussianPRF.html#photutils.psf.CircularGaussianPRF)

It also includes image-based PSF models:

* [ImagePSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.ImagePSF.html#photutils.psf.ImagePSF)
* [GriddedPSFModel](https://photutils.readthedocs.io/en/latest/api/photutils.psf.ImagePSF.html#photutils.psf.ImagePSF)

In [ ]:
flux = 71.4
x_0 = 24.3
y_0 = 25.2
models = [
    CircularGaussianPSF(flux=flux, x_0=x_0, y_0=y_0, fwhm=10.1),
    GaussianPSF(flux=flux, x_0=x_0, y_0=y_0, x_fwhm=10.1, y_fwhm=5.82, theta=21.7),
    MoffatPSF(flux=flux, x_0=x_0, y_0=y_0, alpha=5.1, beta=3.2),
    AiryDiskPSF(flux=flux, x_0=x_0, y_0=y_0, radius=5),
]

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
ax = ax.flatten()
yy, xx = np.mgrid[0:51, 0:51]
for i, model in enumerate(models):
    data = model(xx, yy)
    norm = simple_norm(data, 'sqrt')
    axim = ax[i].imshow(data, norm=norm)
    ax[i].set_title(model.__class__.__name__)

# WebbPSF

[WebbPSF](https://webbpsf.readthedocs.io/en/latest/) is a Python package that computes simulated PSFs for both JWST and Roman.

Let's use webbpsf to create some JWST and Roman PSFs. Please see the [WebbPSF documentation](https://webbpsf.readthedocs.io/en/latest/) for details.

In [ ]:
insts = ['JWST NIRCam', 'JWST NIRISS', 'JWST MIRI', 'Roman WFI']
filts = ['F210M', 'F380M', 'F1000W', 'F087']
oversample = 4
fov_arcsec = 5

psfs = []

nircam = webbpsf.NIRCam()
nircam.filter = filts[0]
psfs.append(nircam.calc_psf(oversample=oversample, fov_arcsec=fov_arcsec))

niriss = webbpsf.NIRISS()
niriss.filter = filts[1]
psfs.append(niriss.calc_psf(oversample=oversample, fov_arcsec=fov_arcsec))

miri = webbpsf.MIRI()
miri.filter = filts[2]
psfs.append(miri.calc_psf(oversample=oversample, fov_arcsec=fov_arcsec))

wfi = webbpsf.WFI()
wfi.filter = filts[3]
psfs.append(wfi.calc_psf(oversample=oversample, fov_arcsec=fov_arcsec))

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
ax = ax.flatten()
for i, (inst, filt, psf) in enumerate(zip(insts, filts, psfs)):
    data = psf[0].data  # oversampled, non-distorted PSF
    norm = simple_norm(data, 'log', vmin=1e-7, vmax=0.1, log_a=1e6)
    ax[i].imshow(data, norm=norm)
    ax[i].set_title(f'{inst} {filt}')

In [ ]:
psfs[0].info()

In [ ]:
webbpsf.display_psf(psfs[2], ext='DET_DIST')

## Create a Photutils `GriddedPSFModel` with `webbpsf`.

A gridded PSF model is a grid of position-dependent ePSFs that takes into account the PSF varying across the detector.

Let's use `webbpsf` to create a gridded PSF model for JWST/NIRCam F200W in detector NRCA1.

In [ ]:
# How to calculate the GriddedPSFModel file that we load in the next cell
# nrc = webbpsf.NIRCam()
# nrc.filter = 'F200W'
# nrc.detector = 'NRCA1'
# psf_model = nrc.psf_grid(num_psfs=16, all_detectors=False, verbose=True, save=True)

In [ ]:
# load the webbpsf GriddedPSF model from the pre-downloaded data
filename = '../data/nircam_nrca1_f200w_fovp101_samp4_npsf16.fits'
psf_model = GriddedPSFModel.read(filename)
psf_model

In [ ]:
# the default oversampling = 4
psf_model.oversampling

In [ ]:
# psf_model contains a 3D cube of PSFs
# 16 2D PSFs each 404 x 404 pixels
psf_model.data.shape

In [ ]:
fig = psf_model.plot_grid(vmax_scale=0.1, figsize=(9, 9))

In [ ]:
fig = psf_model.plot_grid(deltas=True, figsize=(9, 9))

# Let's use this PSF model to create an image of simulated stars

We'll use the `photutils.psf` [make_psf_model_image](https://photutils.readthedocs.io/en/latest/api/photutils.psf.make_psf_model_image.html) function.
### 500 stars in a 2048 x 2048 image (NIRCam SW A1)

In [ ]:
n_sources = 500
shape = (2048, 2048)
data, true_params = make_psf_model_image(shape, psf_model, n_sources,
                                         flux=(500, 20_000), min_separation=25,
                                         seed=0, progress_bar=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
norm = simple_norm(data, 'sqrt', percent=98)
axim = ax.imshow(data, norm=norm)

Now let's add some Gaussian noise (σ = 0.5) to the image.

In [ ]:
noise = make_noise_image(data.shape, mean=0, stddev=0.5, seed=0)
data += noise
error = np.sqrt(np.abs(data))
fig, ax = plt.subplots(figsize=(10, 10))
norm2 = simple_norm(data, 'sqrt', percent=99)
axim = ax.imshow(data, norm=norm2)

The `true_params` output contains an Astropy table containing the true (x, y, flux) of our artificial sources.

In [ ]:
true_params

# Finding Stars in an Image [(photutils.detection)](https://photutils.readthedocs.io/en/latest/user_guide/detection.html)

Let's use the [IRAFStarFinder class](
https://photutils.readthedocs.io/en/latest/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder)
 to find the stars in the simulated image.

In [ ]:
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
stars = finder(data)

In [ ]:
stars

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(data, norm=norm2)
xypos = zip(stars['xcentroid'], stars['ycentroid'])
aper = CircularAperture(xypos, r=20)
patches = aper.plot(ax=ax, color='red')

# The `PSFPhotometry` class

First, we create the `PSFPhotometry` class instance with a few parameters.

We must input a PSF model, which must be an Astropy `Fittable2DModel`.  Photutils provides several PSF models, including a [`GriddedPSFModel`](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GriddedPSFModel.html#photutils.psf.GriddedPSFModel) for spatially-varying PSFs.

We must also input the `fit_shape` parameter, which defines the region around the center of the PSF that is used for fitting the model.

The `finder` keyword takes a Photutils finder object.  Here, we use the `IRAFStarFinder`.  We also input an `aperture_radius` (in pixels) to estimate the initial source fluxes.

In [ ]:
fit_shape = (5, 5)
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=finder, aperture_radius=5, progress_bar=True)

To perform the PSF fitting, we call the `psfphot` object on the data and optional error array.  The result is an Astropy [QTable](https://docs.astropy.org/en/latest/table/index.html) with the fit results.

In [ ]:
phot = psfphot(data, error=error)
phot

We can also use the `make_model_image` method to create a PSF model image of the results.

In [ ]:
model_img = psfphot.make_model_image(data.shape)

fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(model_img, norm=norm)

We can use the `make_residual_image` method to create a residual image.

In [ ]:
resid = psfphot.make_residual_image(data)

fig, ax = plt.subplots(figsize=(10, 10))
norm3 = simple_norm(data, 'sqrt', percent=95)
axim = ax.imshow(resid, norm=norm3)

Our residual image is just noise without any sources, which indicates excellent PSF model fits.

## Comparing results

Let's use our knowledge of the true (x, y) positions and fluxes to compare to our PSF fit results.

We first need to match the table catalogs.

In [ ]:
# convenience function to match (x, y) positions
def xymatch_catalogs(ref_params, params):
    refcat = QTable()
    refcat['TPx'] = ref_params['x_0']
    refcat['TPy'] = ref_params['y_0']
    fitcat = QTable()
    fitcat['TPx'] = params['x_fit']
    fitcat['TPy'] = params['y_fit']
    match = XYXYMatch(separation=1)
    ref_idx, fit_idx = match(refcat, fitcat)

    return ref_params[ref_idx], params[fit_idx]

In [ ]:
true_params, fit_params = xymatch_catalogs(true_params, phot)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
ax[0].plot(true_params['x_0'], fit_params['x_fit'], '.')
ax[0].set_xlabel('x True')
ax[0].set_ylabel('x Fit')
ax[1].plot(true_params['y_0'], fit_params['y_fit'], '.')
ax[1].set_xlabel('y True')
ax[1].set_ylabel('y Fit')
ax[2].plot(true_params['flux'], fit_params['flux_fit'], '.')
ax[2].set_xlabel('Flux True')
ax[2].set_ylabel('Flux Fit')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
pdiff = (true_params['flux'] - fit_params['flux_fit']) / true_params['flux'] * 100.0
ax.hist(pdiff, bins=50)
text = ax.set_xlabel('True/Fit Flux Percent Difference')

# Source Grouping

Source grouping is an optional feature that allows you to group close stars that should be fit simultaneously.

To turn it on, create a [SourceGrouper](https://photutils.readthedocs.io/en/latest/api/photutils.psf.SourceGrouper.html#photutils.psf.SourceGrouper) instance and input it via the grouper keyword. Here we’ll group stars that are within 20 pixels of each other:

In [ ]:
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
stars = finder(data)
stars[0:5]

In [ ]:
min_separation = 35
grouper = SourceGrouper(min_separation)

In [ ]:
x = np.array(stars['xcentroid'])
y = np.array(stars['ycentroid'])
group_ids = grouper(x, y)

Groups is an array with 500 elements (1 per input (x, y) position) with the group IDS.

In [ ]:
group_ids

The grouping algorithm separated the 500 stars into 428 distinct groups

In [ ]:
print(max(group_ids))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
norm4 = simple_norm(data, 'sqrt', percent=99)
ax.imshow(data, norm=norm4, cmap='Greys_r')
cmap = make_random_cmap(ncolors=500, seed=123)
for i in np.arange(1, max(group_ids) + 1):
    mask = group_ids == i
    xypos = zip(x[mask], y[mask])
    ap = CircularAperture(xypos, r=20)
    ap.plot(color=cmap.colors[i], lw=2)

In [ ]:
fit_shape = (5, 5)
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
min_separation = 35
grouper = SourceGrouper(min_separation)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=finder, grouper=grouper, 
                        aperture_radius=5, progress_bar=True)
phot = psfphot(data, error=error)
phot

# Fitting a single source

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(data, norm=norm2)

x = 743
y = 1044
aper = CircularAperture((x, y), r=20)
patches = aper.plot(color='red')

In [ ]:
init_params = QTable()
init_params['x'] = [x]
init_params['y'] = [y]

fit_shape = (5, 5)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=None, 
                        aperture_radius=5, progress_bar=True)
phot = psfphot(data, error=error, init_params=init_params)
phot

In [ ]:
resid = psfphot.make_residual_image(data)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
norm = simple_norm(data, 'sqrt', percent=99)
ax[0].imshow(data, norm=norm)
ax[1].imshow(data - resid, norm=norm)
im = ax[2].imshow(resid, norm=norm)
ax[0].set_title('Data')
aper.plot(ax=ax[0], color='red')
ax[1].set_title('Model')
aper.plot(ax=ax[1], color='red')
ax[2].set_title('Residual Image')
aper.plot(ax=ax[2], color='red')
plt.tight_layout()

# Futher Reading

Please consult the [PSF Photometry documentation](https://photutils.readthedocs.io/en/stable/user_guide/psf.html) for additional features, including:

- Forced Photometry
- Bounded Model Parameters
- Iterative PSF Photometry
- Local Background Subtraction

# Exercise

Start with the simulated image created in the following cell.

In [ ]:
psf_model = CircularGaussianPSF(fwhm=3.1)
n_sources = 500
shape = (1024, 1024)
data, true_params = make_psf_model_image(shape, psf_model, n_sources,
                                         flux=(500, 20_000), min_separation=25,
                                         seed=0, progress_bar=True)
noise = make_noise_image(data.shape, mean=0, stddev=1, seed=0)
data += noise
error = np.sqrt(np.abs(data))
fig, ax = plt.subplots(figsize=(10, 10))
norm2 = simple_norm(data, 'sqrt', percent=99)
axim = ax.imshow(data, norm=norm2)

1. Use the defined `CircularGaussianPSF` PSF model (with FWHM=3.1) to perform PSF photometry of all 500 sources in the image.
2. Plot the residual image.
3. Identify the 3 brightest sources in the image. What are their (x, y) positions and fluxs?
4. Bonus: Plot the image with a red circle around these 3 brightest sources.

# Exercise Solutions

To load the solutions, first uncomment the cells below. Then, run each cell below **twice** (once to load the solution and once more to run the code).

In [ ]:
# %load psf_part1_solution.py

In [ ]:
# %load psf_part2_solution.py

In [ ]:
# %load psf_part3_solution.py

In [ ]:
# %load psf_part4_solution.py